In [1]:
import os
import pyopencl as pcl
import numpy as np

# let's try to find available devices
platforms = pcl.get_platforms()
for p in platforms:
    devs = p.get_devices()
    for d in devs:
        print(d.name, pcl.device_type.to_string(d.type), d.global_mem_size / 10**9)

# let's select the AMD radeon card in this case
dev=None
for p in pcl.get_platforms():
    devs = p.get_devices()
    for d in devs:
        if pcl.device_type.to_string(d.type) == 'GPU' and (d.global_mem_size / 10**9) > 2.0:
            dev = d
            
# make the opencl context
# cntx = pcl.create_some_context()
cntx = pcl.Context(devices=[dev])
queue = pcl.CommandQueue(cntx, device=dev)

Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz CPU 17.179869184
Iris Pro GPU 1.610612736
AMD Radeon R9 M370X Compute Engine GPU 2.147483648


In [2]:
ktest_cl_file = os.path.join('..', 'src', 'cl', 'kernel_tests.cl')
os.path.isfile(ktest_cl_file)

True

In [135]:
# build the kernel
with open(ktest_cl_file, 'r') as f:
    programs = pcl.Program(cntx, f.read()).build()
    f.seek(0)
    print(f.read())

__kernel void addem(__global float * a, __global float * b, __global float * c)
{

  int i = get_global_id(0);
  c[i] = a[i] + b[i];

}


__kernel void multiplyem(__global float * a, __global float * b, __global float * c)
{
  int i = get_global_id(0);
  c[i] = a[i] * b[i];
}

__kernel void testdot(__global float * a, __global float * b, __global float * c){
  int gid = get_global_id(0);
  c[gid] = dot(a[gid], b[gid]);
}

__kernel void test_rowaverage(__global float * in, __global float * out, const int nrows, const int ncols)
{
  float nrowsf = (float) nrows;
  for(int i = 0; i < nrows; i++){
    for (int j = 0; j < ncols; j++){
      out[j] += in[i * ncols + j];
      out[j] /= nrowsf;
    }
  }

}

__kernel void test_reduction_avg(__global float * in, __global float * out, __local float * partial_sums, const int nrows)
{
  int lid = get_local_id(0);
  int gid = get_global_id(0);
  int group_size = get_local_size(0);
  float nrowsf = (float) nrows;

  partial_sums[lid] = in[gid];
  b

In [ ]:
# set up the buffers and arrays
a = np.ones(shape=(10, ), dtype=np.float32) * 3
# b = np.ones(shape=(10, ), dtype=np.float32) * 5
b = np.arange(0,10,1, dtype=np.float32)
c = np.zeros(shape=(10, ), dtype=np.float32)

In [ ]:
a, b, c

In [ ]:
a_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=a)
b_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=b)
c_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=c)

In [ ]:
# queue up the two kernels 
# add_event = programs.addem(queue,
#                            a.shape,
#                            None,
#                            a_buf, 
#                            b_buf,
#                            c_buf)

# multiply_event = programs.multiplyem(queue,
#                                      a.shape,
#                                      None,
#                                      c_buf,
#                                      a_buf,
#                                      b_buf)

dot_event = programs.testdot(queue,
                             a.shape,
                             None,
                             a_buf,
                             b_buf,
                             c_buf)

In [ ]:
# queue.finish()

In [ ]:
# add_event.wait()
# multiply_event.wait()

In [ ]:
pcl.enqueue_copy(queue, b, b_buf)
pcl.enqueue_copy(queue, c, c_buf)

In [ ]:
b

In [ ]:
c

In [ ]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]], dtype=np.float32)
x_avg = np.zeros(shape=(X.shape[1],), dtype=np.float32)

In [ ]:
c

In [ ]:
X_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=X)
x_avg_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=x_avg)

In [ ]:
x_avg

In [ ]:
row_avg_event = programs.test_rowaverage(queue,
                                         X.shape,
                                         None,
                                         X_buf, 
                                         x_avg_buf,
                                         np.int32(X.shape[0]),
                                         np.int32(X.shape[1]))

In [ ]:
pcl.enqueue_copy(queue, x_avg, x_avg_buf)

In [ ]:
x_avg

In [ ]:
X.mean(axis=0)

In [ ]:
X

In [176]:
# z = np.array([1,2,3,4,5,6,7,8,9,10], dtype=np.float32)
z = np.random.normal(size=(2**16,)).astype(np.float32)
z_out = np.zeros(shape=z.shape, dtype=np.float32)

In [177]:
next_pow2_size=16

In [181]:
z_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=z, size=z.nbytes)
z_out_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_WRITE | pcl.mem_flags.COPY_HOST_PTR, hostbuf=z_out)
partial_buf = pcl.LocalMemory(size=z.nbytes)

In [182]:
z.shape

(65536,)

In [192]:
queue.finish()

In [199]:
reduction_event = programs.test_reduction_avg(queue,
                                              z.shape,
                                              None,
                                              z_buf,
                                              z_out_buf,
                                              partial_buf,
                                              np.int32(z.shape[0]))

RuntimeError: clEnqueueNDRangeKernel failed: OUT_OF_RESOURCES

In [194]:
queue.finish()

In [195]:
pcl.enqueue_copy(queue, z_out, z_out_buf).wait()

In [196]:
z_out

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [197]:
z.sum()

-129.96548

In [198]:
z.mean()

-0.0019831159

In [146]:
nrows = int(2**9)
y = np.random.normal(size = (nrows, 6), loc=10.0).astype(np.float32)
y_row_avg = np.zeros(shape=(6,)).astype(np.float32)

In [147]:
y.nbytes

12288

In [148]:
y_buf = pcl.Buffer(cntx, pcl.mem_flags.READ_ONLY | pcl.mem_flags.COPY_HOST_PTR, hostbuf=y)
y_out_buf = pcl.Buffer(cntx, pcl.mem_flags.WRITE_ONLY, size=y_row_avg.nbytes)
partial_sum_buf = pcl.LocalMemory(size=y.nbytes)

In [149]:
row_reduction_event = programs.test_reduction_avg_matrix(queue,
                                                         y.shape,
                                                         None,
                                                         y_buf,
                                                         y_out_buf,
                                                         partial_sum_buf,
                                                         np.int32(y.shape[0]),
                                                         np.int32(y.shape[1]))

In [150]:
row_reduction_event.wait()

In [151]:
pcl.enqueue_copy(queue, y_row_avg, y_out_buf, wait_for=[row_reduction_event])

In [152]:
y_row_avg

array([ 4.97654247,  5.01037407,  5.0156889 ,  5.06041002,  4.98361921,
        5.00504589], dtype=float32)

In [153]:
y.mean(axis=0), y_row_avg

(array([  9.98460674,  10.00198746,  10.04318142,  10.05097008,
          9.94996071,  10.02250481], dtype=float32),
 array([ 4.97654247,  5.01037407,  5.0156889 ,  5.06041002,  4.98361921,
         5.00504589], dtype=float32))

In [154]:
y.nbytes * 2

24576

In [155]:
y.mean(axis=0)

array([  9.98460674,  10.00198746,  10.04318142,  10.05097008,
         9.94996071,  10.02250481], dtype=float32)

In [200]:
# from: https://github.com/pyopencl/pyopencl/blob/master/examples/benchmark.py

In [206]:
from __future__ import print_function
from __future__ import absolute_import
import pyopencl as cl
import numpy
import numpy.linalg as la
import datetime
from time import time

data_points = 2**24 # ~8 million data points, ~32 MB data
workers = 2**6 # 256 workers, play with this to see performance differences
               # eg: 2**0 => 1 worker will be non-parallel execution on gpu
               # data points must be a multiple of workers

a = numpy.random.rand(data_points).astype(numpy.float32)
b = numpy.random.rand(data_points).astype(numpy.float32)
c_result = numpy.empty_like(a)

# Speed in normal CPU usage
time1 = time()
c_temp = (a+b) # adds each element in a to its corresponding element in b
c_temp = c_temp * c_temp # element-wise multiplication
c_result = c_temp * (a/2.0) # element-wise half a and multiply
time2 = time()

print("Execution time of test without OpenCL: ", time2 - time1, "s")


for platform in cl.get_platforms():
    for device in platform.get_devices():
        print("===============================================================")
        print("Platform name:", platform.name)
        print("Platform profile:", platform.profile)
        print("Platform vendor:", platform.vendor)
        print("Platform version:", platform.version)
        print("---------------------------------------------------------------")
        print("Device name:", device.name)
        print("Device type:", cl.device_type.to_string(device.type))
        print("Device memory: ", device.global_mem_size//1024//1024, 'MB')
        print("Device max clock speed:", device.max_clock_frequency, 'MHz')
        print("Device compute units:", device.max_compute_units)
        print("Device max work group size:", device.max_work_group_size)
        print("Device max work item sizes:", device.max_work_item_sizes)

        # Simnple speed test
        ctx = cl.Context([device])
        queue = cl.CommandQueue(ctx, 
                properties=cl.command_queue_properties.PROFILING_ENABLE)

        mf = cl.mem_flags
        a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
        b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
        dest_buf = cl.Buffer(ctx, mf.WRITE_ONLY, b.nbytes)

        prg = cl.Program(ctx, """
            __kernel void sum(__global const float *a,
            __global const float *b, __global float *c)
            {
                        int gid = get_global_id(0);
                        float a_temp;
                        float b_temp;
                        float c_temp;
                        a_temp = a[gid]; // my a element (by global ref)
                        b_temp = b[gid]; // my b element (by global ref)
                        
                        c_temp = a_temp+b_temp; // sum of my elements
                        c_temp = c_temp * c_temp; // product of sums
                        c_temp = c_temp * (a_temp/2.0f); // times 1/2 my a
                        c[gid] = c_temp; // store result in global memory
                }
                """).build()

        global_size=(data_points,)
        local_size=(workers,)
        preferred_multiple = cl.Kernel(prg, 'sum').get_work_group_info( \
            cl.kernel_work_group_info.PREFERRED_WORK_GROUP_SIZE_MULTIPLE, \
            device)

        print("Data points:", data_points)
        print("Workers:", workers)
        print("Preferred work group size multiple:", preferred_multiple)

        if (workers % preferred_multiple):
            print("Number of workers not a preferred multiple (%d*N)." \
                    % (preferred_multiple))
            print("Performance may be reduced.")

        exec_evt = prg.sum(queue, global_size, None, a_buf, b_buf, dest_buf)
        exec_evt.wait()
        elapsed = 1e-9*(exec_evt.profile.end - exec_evt.profile.start)

        print("Execution time of test: %g s" % elapsed)

        c = numpy.empty_like(a)
        cl.enqueue_read_buffer(queue, dest_buf, c).wait()
        equal = numpy.all( c == c_result)

        if not equal:
                print("Results doesn't match!!")
        else:
                print("Results OK")

Execution time of test without OpenCL:  0.13441205024719238 s
Platform name: Apple
Platform profile: FULL_PROFILE
Platform vendor: Apple
Platform version: OpenCL 1.2 (Nov  1 2016 21:34:57)
---------------------------------------------------------------
Device name: Intel(R) Core(TM) i7-4870HQ CPU @ 2.50GHz
Device type: CPU
Device memory:  16384 MB
Device max clock speed: 2500 MHz
Device compute units: 8
Device max work group size: 1024
Device max work item sizes: [1024, 1, 1]
Data points: 16777216
Workers: 64
Preferred work group size multiple: 1
Execution time of test: 0.0348087 s
Results OK
Platform name: Apple
Platform profile: FULL_PROFILE
Platform vendor: Apple
Platform version: OpenCL 1.2 (Nov  1 2016 21:34:57)
---------------------------------------------------------------
Device name: Iris Pro
Device type: GPU
Device memory:  1536 MB
Device max clock speed: 1200 MHz
Device compute units: 40
Device max work group size: 512
Device max work item sizes: [512, 512, 512]
Data points:

/Users/antalek/anaconda/envs/ENVPy3/lib/python3.5/site-packages/ipykernel/__main__.py:91: DeprecationWarning: 'enqueue_read_buffer' has been deprecated in version 2011.1. Please use enqueue_copy() instead.


Results OK
Platform name: Apple
Platform profile: FULL_PROFILE
Platform vendor: Apple
Platform version: OpenCL 1.2 (Nov  1 2016 21:34:57)
---------------------------------------------------------------
Device name: AMD Radeon R9 M370X Compute Engine
Device type: GPU
Device memory:  2048 MB
Device max clock speed: 300 MHz
Device compute units: 10
Device max work group size: 256
Device max work item sizes: [256, 256, 256]
Data points: 16777216
Workers: 64
Preferred work group size multiple: 64
Execution time of test: 0.00768416 s
Results OK
